In [1]:
import json
import os

import sys
from transformers import DonutProcessor

PUBTABLES_ANN = '../../pubtables-1m/Donut_Annotations_PT/minimal/Val/'
PUBTABLES_IMG = '../../pubtables-1m/PubTables-1M-Structure/Val/'

PUBTABLES_ANN = '../../pubtables-1m/Donut_Annotations_PT/minimal/Val/'
PUBTABLES_IMG = '../../pubtables-1m/PubTables-1M-Structure/Val/'

PUBTAB_ANN = '../../pubtabnet/anns/train/'
PUBTAB_IMG = '../../pubtabnet/imgs/train/'

ANN_PATH = PUBTAB_ANN
IMAGE_PATH = PUBTAB_IMG
IMG_FORMAT = '.png'

with open(ANN_PATH[:-1] + '_trunc_filelist.json') as file:
    json_list = file.read().splitlines()
    
json_list = [item for item in json_list]

2023-11-01 23:10:29.798159: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 23:10:29.798185: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 23:10:29.798203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 23:10:29.802735: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 23:10:30.348464: W tensorflow/compiler/

In [2]:
aux_list = []

for json_item in json_list:
    aux_list.append(json_item[:-5])

json_list = aux_list

In [3]:
json_list[:10]

['PMC6051241_006_00',
 'PMC3398398_014_00',
 'PMC2682793_006_00',
 'PMC6082926_006_00',
 'PMC2809502_009_00',
 'PMC3890642_004_00',
 'PMC3926592_006_01',
 'PMC548940_003_00',
 'PMC4394591_006_00',
 'PMC1906827_006_00']

In [4]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers import VisionEncoderDecoderConfig
import torch

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")

image_size = [720, 720]
max_length = 1500#config.decoder.max_position_embeddings


config.encoder.image_size = image_size
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
from transformers import AddedToken

new_tokens = ["<table_extraction>", "<table>", "<row>", "<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
new_tokens += ["<content_row_and_col_header>", "<content_row_header>", "<content_col_header>", "<content>"]
new_tokens += ["1", "≥","≤"]
new_tokens += ["<b>", "</b>", "<i>", "</i>", "<sup>", "</sup>","<sub>", "</sub>"]

for i in range(2):
    for j in range(2):
        for k in range(2):
            new_tokens.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            new_tokens.append("<span_type=1" + str(i) + str(j) + str(k) + ">")

new_tokens = [AddedToken(tag, rstrip = True, lstrip=True, normalized=False) for tag in new_tokens]

processor.tokenizer.add_tokens(new_tokens, special_tokens = False)

38

In [6]:
cell_types = ["<cell>", "<row_and_col_header>", "<row_header>", "<col_header>"]
for i in range(2):
    for j in range(2):
        for k in range(2):
            cell_types.append("<span_type=0" + str(i) + str(j) + str(k) + ">")
            cell_types.append("<span_type=1" + str(i) + str(j) + str(k) + ">")


cell_tokens = [processor.tokenizer.convert_tokens_to_ids([cell_type])[0] for cell_type in cell_types]
row_tokens = [processor.tokenizer.convert_tokens_to_ids([row_type])[0] for row_type in ['<row>', '</s>']]

In [7]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config=config, ignore_mismatched_sizes=True)

In [8]:
from torch import nn
from typing import Optional, Tuple, List, Union
from collections import defaultdict 

class DimPositionalEmbedding(nn.Module):
    """
    This module learns n sets positional embeddings up to n fixed maximum sizes.
    """

    def __init__(self, offset, dim_counters, max_dim_lens, embeddings_d):
        super().__init__()
        
        self.offset = offset
        self.embeddings_d = embeddings_d
        self.counter_dim = len(dim_counters)+1
        
        embeddings_list = []
        for i in range(len(max_dim_lens)):
            embeddings_list.append(nn.Embedding(max_dim_lens[i], embeddings_d))
        
        self.embeddings = nn.ModuleList(embeddings_list)
        
        self.counter_keys = defaultdict(lambda: 0) 
        for i in range(self.counter_dim-1):
            for counter_key in dim_counters[i]:
                self.counter_keys[counter_key] = i+1

                
    def get_sequence_embeddings_map(self, input_ids:torch.Tensor, counters_state:List = None):
        seq_len = input_ids.shape[0]
        embeddings_map = [[0 for i in range(seq_len)] for j in range(self.counter_dim)]
        
        if counters_state == None:
            counters = [0 for i in range(self.counter_dim)]
            counters[0] = self.offset
        else:
            counters = counters_state
        
        for i, token in enumerate(input_ids):
            counter = self.counter_keys[int(token)]
            for j in range(counter):
                counters[j] = 0
            counters[counter] += 1
            
            for j in range(self.counter_dim):
                embeddings_map[j][i] = counters[j]
            
        return embeddings_map, counters
    
    
    def get_embeddings_sum(self, index_lists:List):
        embeddings_sum = torch.zeros(self.embeddings_d, device = self.embeddings[0].weight.device)
        for i, index_list in enumerate(index_lists):
            partial_embedding = self.embeddings[i](torch.as_tensor(index_list, device = self.embeddings[0].weight.device))
            embeddings_sum = torch.add(embeddings_sum, partial_embedding)
        
        return embeddings_sum
    
    
    def forward(self, input_ids: torch.Tensor, counters_states:List = None):
        new_counters_states = []
        pos_embeddings = []
        
        for i, sequence in enumerate(input_ids):
            if counters_states == None or counters_states == 0:
                embeddings_map, new_counters_state = self.get_sequence_embeddings_map(sequence, None)
            else:
                embeddings_map, new_counters_state = self.get_sequence_embeddings_map(sequence, counters_states[i])
            
            pos_embeddings.append(self.get_embeddings_sum(embeddings_map))
            new_counters_states.append(new_counters_state)
           
        
        return torch.stack(pos_embeddings)


In [9]:
model.decoder.model.decoder.embed_positions = DimPositionalEmbedding(2, [cell_tokens, row_tokens], [2048, 200, 200], 1024)

In [10]:
import sys
sys.path.insert(0, '../src')

from modeling_tabeleiro import TabeleiroModel

model = TabeleiroModel.from_pretrained("tabeleiro/")

You are using a model of type dimbart to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.


In [11]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
processor.image_processor.do_align_long_axis = False


In [12]:
with open("msg.json", 'w') as out:
        json.dump({'outputs': []}, out, ensure_ascii=False, indent=4)

def write_msg(msg):
    with open("msg.json", encoding="utf-8") as f:
        json_data = json.load(f)
    
    with open("msg.json", 'w') as out:
        json_data['outputs'].append(msg)
        json.dump(json_data, out, ensure_ascii=False, indent=4)

In [13]:
def cel2token(cell):
    if cell['span_type'][10:] != '0000':
        sequence = "<" + cell['span_type'] + ">"
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<content_row_and_col_header>"
            elif cell['col_header']:
                sequence += "<content_col_header>"
            elif cell['row_header']:
                sequence += "<content_row_header>"
            else:
                sequence += "<content>"
            sequence += cell['content']
    else:
        sequence = ""
        if cell['content_holder']:
            if cell['row_header'] and cell['col_header']:
                sequence += "<row_and_col_header>"
            elif cell['col_header']:
                sequence += "<col_header>"
            elif cell['row_header']:
                sequence += "<row_header>"
            else:
                sequence += "<cell>"
            sequence += cell['content']
        
    return sequence

def row2token(row):
    sequence = "<row>"
    for cell in row:
        sequence += cel2token(cell)
    
    return sequence


def table2token(table):
    sequence = "<table>"
    for row in table:
        sequence += row2token(row)
    
    return sequence


def json2token(json):
    sequence = ""
    if('tables' in json):
        for table in json['tables']:
            sequence += table2token(table)

    return sequence

In [14]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        annotations,
        image_size,
        max_length,
        shuffle = True,
        split = "train",
        ignore_id = -100,
        prompt_end_token = None,
    ):            
        self.annotations = annotations
        
        
        self.image_size = image_size
        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        
        file_name = self.annotations[idx]
        
        with open(ANN_PATH + file_name + ".json", encoding="utf-8") as f:
            annotation = json.load(f)
        
        image = Image.open(IMAGE_PATH + file_name + IMG_FORMAT)
        
        
        # inputs
        pixel_values = processor(image.convert("RGB"), random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()
        pixel_values = pixel_values.squeeze()
        
        target_sequence = json2token(annotation)+"</s>"
        
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length= max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id
        
        
        encoding = dict(pixel_values=pixel_values,
                        labels=labels)
        
        return encoding

In [15]:
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(["<table_extraction>"])[0] 


In [16]:
train_dataset = DonutTableDataset(json_list,
                             max_length = max_length,
                             image_size = image_size)

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [20]:
#model = torch.load("model_epoch_checkpoint_.bin")
start_epoch = 0
avg_size = 100

In [ ]:
import torch
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-5)


for epoch in range(start_epoch, 1):
    print("Epoch:", epoch+1)
    mean_loss = 0
    mean_smpl_loss = 0 
    model.train()
    for i, batch in enumerate(tqdm(train_dataloader)):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        
        pixel_values = batch["pixel_values"]
        labels = batch["labels"]
        
        
        outputs = model(pixel_values=pixel_values, labels=labels)
        
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        mean_loss += loss.item()   
        mean_smpl_loss += loss.item() 
        if i % avg_size == 0:
            print(str(i) + " Loss: ", mean_smpl_loss/avg_size)
            write_msg("batch " + str(i) +" loss: "+ str(mean_smpl_loss/avg_size))
            mean_smpl_loss = 0 
        
        if i % 10000 == 0:
             model.save_pretrained("../../pubtabnet/TML-lre-5-checkpoint")
    
    model.save_pretrained("../../pubtabnet/TML-lre-5-checkpoint")
    print("Epoch's mean loss: ", mean_loss/len(train_dataloader))
    
    write_msg("Epoch checkpointed: " + str(epoch+1) +" \n"+
              "Epoch's mean Loss: " + str(mean_loss/len(train_dataloader)))

Epoch: 1


  0%|          | 0/125195 [00:00<?, ?it/s]

0 Loss:  0.0003856056928634644
<table><row><col_header><b> Ossification centers</b><col_header><b> I group (n =5)</b><col_header><b> II group (n = 4)</b><col_header><b> III group (n = 2)</b><col_header><b> IV group (n = 2)</b><row><cell><i> Caput humeri</i><cell> OCT-0 (2)<cell> OCT-0 (2) OCT-2 (2)<cell> OCT-2 (1 ) OCT-3 (1 )<cell> OCT-3 (2)<row><cell><i> Trochlea humeri</i><cell><cell><cell> OCT-0 (1 )<cell> OCT-3 (2)<row><cell><i> Capitulum humeri</i><cell><cell><cell> OCT-1 (1 )<cell> OCT-0 (2)<row><cell><i> Caput radii</i><cell><cell><cell><cell> OCT-2 (2)<row><cell><i> Trochlea radii</i><cell><cell><cell><cell> OCT-2 (2)<row><cell><i> Os carpi intermedium</i><cell><cell><cell><cell> OCT-2 (2)<row><cell><i> Os carpi radiale</i><cell><cell><cell><cell> OCT-2 (1 )<row><cell><i> Os carpi ulnare</i><cell><cell><cell><cell> OCT-1 (2)<row><cell><i> Os carpi accessorium</i><cell><cell> OCT-1 (1 )<cell> OCT-2 (1 )<cell> OCT-3 (2)<row><cell><i> Os pubis</i><cell> OCT-2 (3) OCT-0 (1 )<cell> 

<table><row><col_header><col_header><b> Burri et al.</b><col_header><b> Charles et al.</b><col_header><b> Frese et al.</b><col_header><b> Nielsen et al. (2001 )</b><col_header><b> Nielsen et al. (2004)</b><col_header><b> Okkes et al.</b><row><span_type=1000><span_type=1001><span_type=1001><span_type=1001><content><i> Domain A: Selection of patients and GPs (refers to all studies regardless the review question)</i><span_type=1001><span_type=1001><span_type=0001><row><cell> Was the symptom to be investigated clearly described?<cell> no<cell> no<cell> no<cell> no<cell> no<cell> no<row><cell> Were the selection criteria of the patients clearly described?<cell> yes<cell> yes<cell> yes<cell> unclear<cell> unclear<cell> yes<row><cell> Was a consecutive or random sample of patients enrolled?<cell> yes<cell> yes<cell> yes<cell> yes<cell> yes<cell> yes<row><cell> Was it a multi-centre study?<cell> yes<cell> yes<cell> yes<cell> yes<cell> yes<cell> yes<row><cell> Did the selection criteria of the 

100 Loss:  0.06990530848503113
<table><row><col_header><b> No. of Marker</b><col_header><b> Random</b><col_header><b> Half-sib</b><col_header><b> Sib</b><row><cell>1<cell> 3.20 ×1 0<sup> −2</sup><cell> 8.09 ×1 0<sup> −2</sup><cell> 3.09 ×1 0<sup> −1</sup><row><cell> 2<cell>1.22 ×1 0<sup> −3</sup><cell> 7.33 ×1 0<sup> −2</sup><cell> 9.77 ×1 0<sup> −2</sup><row><cell> 3<cell> 4.76 ×1 0<sup> −5</sup><cell> 6.70 ×1 0<sup> −2</sup><cell> 3.11 ×1 0<sup> −2</sup><row><cell> 4<cell> 2.04 ×1 0<sup> −6</sup><cell> 6.52 ×1 0<sup> −5</sup><cell>1.00 ×1 0<sup> −2</sup><row><cell> 5<cell>1.08 ×1 0<sup> −7</sup><cell> 7.29 ×1 0<sup> −6</sup><cell> 3.37 ×1 0<sup> −3</sup><row><cell> 6<cell> 6.1 0 ×1 0<sup> −9</sup><cell> 8.50 ×1 0<sup> −7</sup><cell>1.1 4 ×1 0<sup> −3</sup><row><cell> 7<cell> 3.49 ×1 0<sup> −1 0</sup><cell>1.01 ×1 0<sup> −7</sup><cell> 3.89 ×1 0<sup> −4</sup><row><cell> 8<cell> 2.30 ×1 0<sup> −11</sup><cell>1.31 ×1 0<sup> −8</sup><cell>1.36 ×1 0<sup> −4</sup><row><cell> 9<cell>1.58 ×1

...

<table><row><col_header><b> Epitope</b><col_header><b> Residues</b><col_header><b> References</b><row><cell> (1 ) CWGELMNLATWVGSNLEDPASRE<cell> 61 –83<cell> Sallberg et al.,1 994 [27]; Salfeld et al.,1 989 [8] and Thermet et al., 2004 [28].<row><cell> (2) CLTFGRETVLEY<cell>1 07–11 8<cell> Collucci et al.,1 988 [29].<row><cell> (3) TPPAYR<cell>1 28–1 33<cell> Sallberg et al.,1 991 [30].<row><cell> (4) PPNAPILSTLPE<cell>1 34–1 45<cell> Takahashi et al., 2001 [31 ]. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

<table><row><col_header><b> GlyRα1</b><col_header><b> EC<sub> 50</sub> (μm)</b><col_header><b> Comparison with WT (<i> P</i> )</b><col_header><b> Comparison with N46K (<i> P</i> )</b><col_header><b><i> n</i></b><row><cell> WT<cell> 41 ± 3<cell> —<cell> —<cell> 20<row><cell> N46K<cell> 372 ± 4<cell> < 0.0001<cell> —<cell>1 9<row><cell> N46Q<cell> 51 ± 4<cell> 0.0766<cell> < 0.0001<cell> 5<row><cell> N46D<cell> 202 ± 51<cell> 0.0253<cell> 0.0208<cell> 6<row><cell> N46E<cell> 272 ± 46<cell> 0.0041<cell> 0.082<cell> 6<row><cell> N46A<cell>1 82 ± 26<cell> 0.0057<cell> 0.001 9<cell> 5<row><cell> N46F<cell> 2260 ± 570<cell> 0.011 5<cell> 0.021 2<cell> 6<row><cell> N46W<cell>1 450 ±1 00<cell> 0.0001<cell> 0.0004<cell> 5<row><cell> N46R<cell> 81 6 ±1 30<cell> 0.001 9<cell> 0.01 89<cell> 6<row><cell> N46C<cell> 330 ±1 6<cell> < 0.0001<cell> 0.048<cell> 6<row><cell> N46S<cell> 221 ± 22<cell> 0.0005<cell> 0.001<cell> 6<row><cell> N46T<cell> 756 ±1 33<cell> 0.003<cell> 0.0343<cell> 6</s><pad><pad><

<table><row><col_header><b><i> Administrative unit</i></b><col_header><b><i> Distance assumption</i></b><span_type=1000><span_type=1001><content_col_header><b><i> Sampling design</i></b><span_type=0001><col_header><b><i> Length of work day assumption</i></b><row><cell><cell><cell><i> 33 × 6 design (in team days)</i><cell><i> 67 × 3 design (in team days)</i><cell><i> 30 × 30 design (in team days)</i><cell><row><cell> Fur Baranga<cell> Actual distance<cell> 8.25<cell>11.1 7<cell> 30.00<cell> 8.5 hr work day<row><cell> Habila<cell> Actual distance<cell> 4.1 3<cell> 5.58<cell>1 5.00<cell>1 2.0 hr work day<row><cell> Fur Baranga<cell> 3× Actual distance<cell>11.00<cell>1 6.75<cell> 30.00<cell> 9.5 hr work day<row><cell> Habila<cell> 3× Actual distance<cell> 8.25<cell>1 3.40<cell> 30.00<cell> 9.0 hr work day<row><cell> Fur Barnaga<cell> 5× Actual distance<cell>11.00<cell> 22.33<cell> 30.00<cell>11.0 hr work day<row><cell> Habila<cell> 5× Actual distance<cell>11.00<cell> 22.33<cell> 30.00<cel

<table><row><col_header><b></b><col_header><b> Teaching method</b><col_header><b> Teaching aim</b><row><cell> [a]<cell> Lecture (including external lecturers)<cell> Knowledge<row><cell> [b]<cell> PowerPoint presentation<cell> Knowledge<row><cell> [c]<cell> Journal club<cell> Knowledge<row><cell> [d]<cell> Writing exercises<cell> Knowledge<row><cell> [e]<cell> Presentations<cell> Knowledge<row><cell> [f]<cell> Music/Songs<cell> Knowledge/Attitude<row><cell> [g]<cell> Individual conversations<cell> Knowledge/Attitude<row><cell> [h]<cell> Spiritual impulses<cell> Knowledge/Attitude<row><cell> [i]<cell> Group discussion/Panel discussion/Small group discussion<cell> Knowledge/Attitude<row><cell> [j]<cell> Videos/Films/Pictures/Symbols<cell> Knowledge/Attitude<row><cell> [k]<cell> Work with biblical/interreligious texts, other readings<cell> Knowledge/Attitude<row><cell> [l]<cell> Visiting patients in patient care units<cell> Knowledge/Attitude<row><cell> [m]<cell> Role plays<cell> Knowledge

<table><row><col_header><b> Cebpb</b><col_header><col_header><b> Esr1</b><col_header><col_header><b> Foxo1</b><col_header><col_header><b> Foxo4</b><col_header><col_header><b> Foxp2</b><col_header><row><cell> GATTGCA<cell> 48<cell> AGTCAAG<cell> 9<cell> ACAAACA<cell> 53<cell> GATAAAC<cell> 30<cell> TGTATAC<cell> 32<row><cell> TGTTGCA<cell> 43<cell> ATGATCT<cell> 7<cell> TGTTATT<cell> 52<cell> AATAAAC<cell> 29<cell> TGTTGTA<cell> 21<row><cell> TATTGCG<cell> 42<cell> ACGTCGA<cell> 6<cell> TGTTTTT<cell> 47<cell> CTATTTA<cell> 27<cell> TGTGTAC<cell> 20<row><cell> CATTGCA<cell> 39<cell> ACGTTCT<cell> 5<cell> TGTTTGT<cell> 44<cell> AATAAAT<cell> 23<cell> CTTGATA<cell>1 2<row><cell> CATTGCG<cell> 38<cell> AGGTGCA<cell> 4<cell> ACAACAT<cell> 41<cell> CATAAAC<cell> 22<cell> AATATCC<cell>1 0<row><cell> GATTGCG<cell> 35<cell> ATGTTCT<cell> 4<cell> TGTTGAT<cell> 41<cell> GATAAAT<cell> 21<cell> TGTGCTT<cell>1 0<row><cell> TTTTGCA<cell> 35<cell> ACGATAT<cell> 3<cell> TGTTATC<cell> 32<cell> CATAAAT<ce

In [ ]:
len(train_dataset)

In [ ]:
model.save_pretrained("../../pubtabnet/model-minimal-3D-epoch1-lr4")

In [ ]:
torch.save(model.state_dict(), "../../pubtabnet/model-minimal-3D-epoch1-pytorch")

In [ ]:
processor.save_pretrained("../../pubtabnet/Donut_PubTables_TML_Processor")

In [ ]:
sys.path.insert(0,'../src/backup/dimbart')

from modeling_dimbart import DiMBartForCausalLM
from configuration_dimbart import DiMBartConfig

In [ ]:
DiMBartConfig()

In [ ]:
decoder_config = model.config.decoder

In [ ]:
decoder_config

In [ ]:
dimBart_config = DiMBartConfig(**decoder_config.__dict__)

In [ ]:
dimbart = model.decoder

In [ ]:
dimbart.config = dimBart_config
dimbart.model.config = dimBart_config
dimbart.model.decoder.config = dimBart_config

In [ ]:
dimbart.model.decoder.config

In [ ]:
dimbart.config.pos_counters = [cell_tokens, row_tokens]

In [ ]:
dimbart.config

In [ ]:
dimbart.save_pretrained("tabeleiro/dimbart_decoder")

In [ ]:
model.encoder.save_pretrained("tabeleiro/donut_encoder")